In [ ]:
import os
import json
import tensorflow as tf
from object_detection.utils import dataset_util

# Pfade definieren
LABELME_DIR = "path_to_labelme_jsons"  # Ordner mit den LabelMe JSON-Dateien
IMAGE_DIR = "path_to_images"          # Ordner mit den zugehörigen Bildern
OUTPUT_TFRECORD = "output.tfrecord"

# Hilfsfunktion zum Erstellen von TFRecord-Features
def create_tf_example(json_file, image_dir):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Bildinformationen
    image_path = os.path.join(image_dir, data['imagePath'])
    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()

    width = data['imageWidth']
    height = data['imageHeight']

    filename = data['imagePath'].encode('utf-8')
    image_format = b'jpeg'  # oder b'png', je nach Bildformat

    # Annotationen (Bounding Boxes und Labels)
    xmins, xmaxs, ymins, ymaxs = [], [], [], []
    classes_text, classes = [], []

    for shape in data['shapes']:
        if shape['shape_type'] == 'rectangle':
            x_min = shape['points'][0][0] / width
            y_min = shape['points'][0][1] / height
            x_max = shape['points'][1][0] / width
            y_max = shape['points'][1][1] / height

            xmins.append(x_min)
            xmaxs.append(x_max)
            ymins.append(y_min)
            ymaxs.append(y_max)

            classes_text.append(shape['label'].encode('utf-8'))
            classes.append(1)  # Klassen-ID, passe dies an deine Kategorien an

    # TFRecord-Feature erstellen
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Hauptfunktion
def labelme_to_tfrecord(labelme_dir, image_dir, output_path):
    writer = tf.io.TFRecordWriter(output_path)

    for json_file in os.listdir(labelme_dir):
        if json_file.endswith('.json'):
            json_path = os.path.join(labelme_dir, json_file)
            tf_example = create_tf_example(json_path, image_dir)
            writer.write(tf_example.SerializeToString())

    writer.close()
    print(f"TFRecord gespeichert: {output_path}")

# Ausführen
labelme_to_tfrecord(LABELME_DIR, IMAGE_DIR, OUTPUT_TFRECORD)
